<a href="https://colab.research.google.com/github/aaradhyabansal/redesigned-octo-lamp/blob/main/salesforecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install Uq upgini catboost

In [ ]:
from os.path import exists
import pandas as pd

df_path="train.csv.zip" if exists("train.csv.zip") else ("https://raw.githubusercontent.com/upgini/upgini/main/notebooks/train.csv.zip")
df=pd.read_csv(df_path)
df=df.sample(19_000,random_state=0)
df["store"]=df["store"].astype(str)
df["item"]=df["item"].astype(str)
df["date"]=pd.to_datetime(df["date"])
df.sort_values("date",inplace=True)
df.reset_index(inplace=True,drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [ ]:
train=df[df["date"]<"2017-01-01"]
test=df[df["date"]>="2017-01-01"]

In [ ]:
train_features=train.drop(columns=["sales"])
train_target=train["sales"]
test_features=test.drop(columns=["sales"])
test_target=test["sales"]

In [ ]:
from upgini import FeaturesEnricher,SearchKey
from upgini.metadata import CVType

enricher=FeaturesEnricher(
    search_keys={
        "date":SearchKey.DATE,
    },
    cv=CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features,test_target)])

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history
Detected task type: ModelTaskType.REGRESSION



Column name,Status,Errors
target,All valid,-
date,All valid,-



Running search request, search_id=ecaa52de-683f-48bb-8665-056b87682525
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com



14 relevant feature(s) found with the search keys: ['date']


,,item,0.604817,100.000000,categorical,
,,store,0.202943,100.000000,categorical,
Upgini,Public data,f_events_date_year_cos1_9014a856,0.041792,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_48_b39cd0c4,0.040530,100.000000,numerical,Free
Upgini,Public data,f_events_date_week_sin1_847b5db1,0.034147,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_31_fa6d9a99,0.032131,100.000000,numerical,Free
Upgini,Public data,f_events_date_week_cos1_f6a8c1fc,0.015611,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_0_1ff14cbf,0.007556,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_34_c3ef5b4f,0.004558,100.000000,numerical,Free
Upgini,Public data,f_financial_date_snp500_3e044492,0.003459,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_33_89bb7578,0.002805,100.000000,numerical,Free


Calculating accuracy uplift after enrichment...


Quality metrics


,Rows,Baseline mean_squared_error,Enriched mean_squared_error,Uplift
,,,,
Train,15213,331.590507,195.156693,136.433814
Eval 1,3787,581.575525,384.384798,197.190727


In [ ]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model=CatBoostRegressor(verbose=False,allow_writing_files=False,random_state=0)

enricher.calculate_metrics(
    train_features,train_target,
    eval_set=[(test_features,test_target)],
    estimator=model,
    scoring="mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...



,Rows,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
,,,,
Train,15213,0.256710,0.157259,0.099451
Eval 1,3787,0.273117,0.190251,0.082866


In [ ]:
enriched_train_features=enricher.transform(train_features,keep_input=True)
enriched_test_features=enricher.transform(test_features,keep_input=True)
enriched_train_features.head()

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=d329ea0f-7607-412e-a286-2f9b71f49552
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


Retrieving selected features from data sources...

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=2580e226-d05c-4e0e-8add-9906e06ad063
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


Retrieving selected features from data sources...



,date,store,item,f_events_date_year_cos1_9014a856,f_weather_date_weather_umap_48_b39cd0c4,f_events_date_week_sin1_847b5db1,f_weather_date_weather_umap_31_fa6d9a99,f_events_date_week_cos1_f6a8c1fc,f_weather_date_weather_umap_0_1ff14cbf,f_weather_date_weather_umap_34_c3ef5b4f,f_financial_date_snp500_3e044492,f_weather_date_weather_umap_33_89bb7578,f_economic_date_cci_pca_3_10646e17,f_financial_date_gold_3295eb6b,f_economic_date_cbpol_pca_2_33d6e3fc,f_financial_date_finance_umap_2_8ebbf71b,f_weather_date_weather_umap_5_88023931
0,2013-01-01,7,5,0.98522,4.540985,0.781831,4.712653,0.62349,3.756893,5.664261,1426.189941,4.644803,2.291145,1674.800049,-0.836517,10.829547,4.586665
1,2013-01-01,4,9,0.98522,4.540985,0.781831,4.712653,0.62349,3.756893,5.664261,1426.189941,4.644803,2.291145,1674.800049,-0.836517,10.829547,4.586665
2,2013-01-01,1,33,0.98522,4.540985,0.781831,4.712653,0.62349,3.756893,5.664261,1426.189941,4.644803,2.291145,1674.800049,-0.836517,10.829547,4.586665
3,2013-01-01,3,41,0.98522,4.540985,0.781831,4.712653,0.62349,3.756893,5.664261,1426.189941,4.644803,2.291145,1674.800049,-0.836517,10.829547,4.586665
4,2013-01-01,5,24,0.98522,4.540985,0.781831,4.712653,0.62349,3.756893,5.664261,1426.189941,4.644803,2.291145,1674.800049,-0.836517,10.829547,4.586665


In [ ]:
model.fit(train_features,train_target)
preds=model.predict(test_features)
eval_metric(test_target.values,preds,"SMAPE")

[37.65141857448004]

In [ ]:
model.fit(enriched_train_features,train_target)
enriched_pred=model.predict(enriched_test_features)
eval_metric(test_target.values,enriched_pred,"SMAPE")

[14.042277164606292]